# GSQL 101

In this tutorial, we will introduce core concepts in creating and querying graph databases using GSQL with pyTigerGraph. We will show you how to create a graph schema, load data, write simple parameterized queries, and run your queries. Further GSQL 101 documentation can be found here (https://docs.tigergraph.com/gsql-ref/current/tutorials/gsql-101/).

## Data set

For this tutorial, we will create and query the simple friendship social graph shown in the previous section. The data for this graph consists of two files in csv (comma-separated values) format.

To follow along with this tutorial, please save these two files, person.csv and friendship.csv, to your local disk.

person.csv

name,gender,age,state<br>
Tom,male,40,ca<br>
Dan,male,34,ny<br>
Jenny,female,25,tx<br>
Kevin,male,28,az<br>
Amily,female,22,ca<br>
Nancy,female,20,ky<br>
Jack,male,26,fl

friendship.csv

person1,person2,date<br>
Tom,Dan,2017-06-03<br>
Tom,Jenny,2015-01-01<br>
Dan,Jenny,2016-08-03<br>
Jenny,Amily,2015-06-08<br>
Dan,Nancy,2016-01-03<br>
Nancy,Jack,2017-03-02<br>
Dan,Kevin,2015-12-30

## Connecting to the Database with pyTigerGraph

In [1]:
import json
import pyTigerGraph as tg

from pyTigerGraph import TigerGraphConnection

# Read in DB configs
with open('../config.json', "r") as config_file:
    config = json.load(config_file)
    
conn = TigerGraphConnection(
    host=config["host"],
    username=config["username"],
    password=config["password"]
)

### Create a graph

Next, use the CREATE GRAPH command to create a graph named Social.

In [2]:
results = conn.gsql(
'''
CREATE GRAPH Social_101 ()
'''
)

print(results)

Semantic Check Fails: The graph name conflicts with another type or existing graph names! Please use a different name.
The graph Social_101 could not be created!


### Connect to graph

In [3]:
conn.graphname="Social_101"

if config["getToken"]: 
    conn.getToken(conn.createSecret())

## Define a Schema for graph Social_101

This section of the tutorial uses a simple example to walk you through the steps to define a schema in GSQL with pyTigerGraph.

### Create a vertex type

Use CREATE VERTEX to define a vertex type named Person. Here, PRIMARY_ID is required: each person must have a unique identifier. The rest is the optional list of attr ibutes which characterize each person vertex, in the format attribute_name data_type, attribute_name data_type, …​

### Create an edge type

Next, use the CREATE …​ EDGE command to create an edge type named Friendship. The keyword UNDIRECTED indicates this edge is a bidirectional edge, meaning that information can flow starting from either vertex. If you’d rather have a unidirectional connection where information flows only from the FROM vertex, use the DIRECTED keyword in place of UNDIRECTED. Here, FROM and TO are required to specify which two vertex types the edge type connects. An individual edge is specified by giving the primary_ids of its source (FROM) vertex and target (TO) vertex. These are followed by an optional list of attributes, just as in the vertex definition.

In [4]:
results = conn.gsql(
    ''' 
    USE GRAPH Social_101
    
    # Create vertex and edge type 
    CREATE SCHEMA_CHANGE JOB social_101_schema FOR GRAPH Social_101 {
        ADD VERTEX Person (PRIMARY_ID name STRING, name STRING, age INT, gender STRING, state STRING);
        ADD UNDIRECTED EDGE Friendship (FROM Person, TO Person, connect_day DATETIME);
    }
    
    RUN SCHEMA_CHANGE JOB social_101_schema
    '''
)

print (results)

Using graph 'Social_101'
Semantic Check Fails: The vertex name Person is used by another object! Please use a different name.
Failed to create schema change jobs: [social_101_schema].


At this point, we have created a Person vertex type, a Friendship edge type, and a Social_101 graph that includes them.

You’ve now built your first graph schema!

## Load Data for graph Social_101

After creating a graph schema, the next step is to load data into it. The task here is to instruct the GSQL loader how to associate ("map") the fields in a set of data files to the attributes in your vertex types and edge types of the graph schema we just defined.

You should have the two data files person.csv and friendship.csv on your local disk. It’s not necessary that they are in the same folder with you.

### Define a loading job

In [5]:
results = conn.gsql(
    '''
    USE GRAPH Social_101

    CREATE LOADING JOB load_social_101 FOR GRAPH Social_101 {
        DEFINE FILENAME person_file;
        DEFINE FILENAME friendship_file;
        
        LOAD person_file
            TO VERTEX Person VALUES ($0, $0, $2, $1, $3)
            USING Header="true",  separator=",";
            
        LOAD friendship_file
            TO EDGE Friendship VALUES ($0, $1, $2)
            USING Header="true", separator=",";
    }
    '''
)

print (results)

Using graph 'Social_101'
Semantic Check Fails: The job name load_social_101 already exists in other objects!
Failed to create loading jobs: [load_social_101].


### Run a loading job

You can now run your loading job to load data into your graph using runLoadingJobWithFile(filePath: str, fileTag: str, jobName: str, sep: str = None) 

* filePath: File variable name or file path for the file containing the data.
* fileTag: The name of file variable in the loading job (DEFINE FILENAME <fileTag>).
* jobName: The name of the loading job.
* sep: Data value separator. If your data is JSON, you do not need to specify this parameter. The default separator is a comma ,.
    
note: change the file path to your local path

In [6]:
uploadPersonFile = conn.runLoadingJobWithFile("person.csv", "person_file", "load_social_101", ",")
print (uploadPersonFile)

uploadFriendshipFile = conn.runLoadingJobWithFile("friendship.csv", "friendship_file", "load_social_101", ",")
print (uploadFriendshipFile)

[{'sourceFileName': 'Online_POST', 'statistics': {'validLine': 8, 'rejectLine': 0, 'failedConditionLine': 0, 'notEnoughToken': 0, 'invalidJson': 0, 'oversizeToken': 0, 'vertex': [{'typeName': 'Person', 'validObject': 7, 'noIdFound': 0, 'invalidAttribute': 1, 'invalidAttributeLines': ['1:age'], 'invalidAttributeLinesData': ['name,gender,age,state\n'], 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}], 'edge': [], 'deleteVertex': [], 'deleteEdge': []}}]
[{'sourceFileName': 'Online_POST', 'statistics': {'validLine': 8, 'rejectLine': 0, 'failedConditionLine': 0, 'notEnoughToken': 0, 'invalidJson': 0, 'oversizeToken': 0, 'vertex': [], 'edge': [{'typeName': 'Friendship', 'validObject': 7, 'noIdFound': 0, 'invalidAttribute': 1, 'invalidAttributeLines': ['1:connect_day'], 'invalidAttributeLinesData': ['person1,person2,date\n'], 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}], 'dele

### Get vertex/edge statistics

In [7]:
vertices = conn.getVertexTypes()
total_count = 0
for vertex in vertices:
    vertex_cnt = conn.getVertexCount(vertex)
    total_count += vertex_cnt
    print("Node count: ({} : {}) ".format(vertex, vertex_cnt))
print("Total node count: ", total_count)

Node count: (Person : 7) 
Total node count:  7


In [8]:
import pprint
edge_count = conn.getEdgeCount()
print("Edges count: total ", sum(edge_count.values()))
pprint.pprint(edge_count) 

Edges count: total  7
{'Friendship': 7}


### Select vertices

If you want to look up the details about a vertex with its primary ID, you can use the following REST call.
Example. Find a Person vertex whose primary_id is "Tom".

In [9]:
import pandas as pd

dan = conn.getVerticesById("Person", "Tom")

print (pd.json_normalize(dan))

  v_id  v_type attributes.name  attributes.age attributes.gender  \
0  Tom  Person             Tom              40              male   

  attributes.state  
0               ca  


### Select edges

In similar fashion, we can see details about edges. To describe an edge, you name the types of vertices and edges in the two parts or three parts of a URL. Example. Find all friendship edges whose source vertex’s primary_id is "Tom".

In [10]:
friendship = conn.getEdges("Person", "Tom", "Friendship")

print (pd.json_normalize(friendship))

       e_type  directed from_id from_type  to_id to_type  \
0  Friendship     False     Tom    Person    Dan  Person   
1  Friendship     False     Tom    Person  Jenny  Person   

  attributes.connect_day  
0    2017-06-03 00:00:00  
1    2015-01-01 00:00:00  


## Develop Parameterized Queries

We just saw how easy and quick it is to run simple built-in queries. However, you’ll undoubtedly want to create more customized or complex queries. GSQL puts maximum power in your hands through parameterized vertex set queries.

Parameterized queries let you traverse the graph from one vertex set to an adjacent set of vertices, again and again, performing computations along the way, with built-in parallel execution and handy aggregation operations. You can even have one query call another query. But we’ll start simple.

A GSQL parameterized query has three steps.

* Define your query in GSQL. This query will be added to the GSQL catalog.

* Install one or more queries in the catalog, generating a REST endpoint for each query.

* Run an installed query, supplying appropriate parameters, either as a GSQL command or by sending an HTTP request to the REST endpoint.

### A simple 1-hop query

Now, let’s write our first GSQL query. We’ll display all the direct (1-hop) neighbors of a person, given as an input parameter.

In [11]:
query = '''
    USE GRAPH Social_101
    
    CREATE QUERY hello (VERTEX<Person> p) {
        start = {p};
        result = SELECT tgt
                 FROM start:s -(Friendship:e)- Person:tgt;
                 
    PRINT result;
    }
'''

This query features one SELECT statement. The query starts by seeding a vertex set start with the person vertex identified by parameter p passed in from the query call. The curly braces tell GSQL to construct a set containing the enclosed items.

Next, the SELECT statement describes a 1-hop traversal according to the pattern described in the FROM clause:

start:s -(Friendship:e)- Person:tgt

The pattern means we select all edges beginning from the given source set (start), which have the given undirected edge type (Friendship) and which end at the given vertex type (Person). The FROM clause defines vertex and edge set aliases using :<alias>:

* s is the alias for the source vertex,

* e is the edge alias

* tgt is the target vertex alias.

Refer back to the initial clause and the assignment (result = SELECT tgt). Here we see the target set’s alias tgt. This means that the SELECT statement returns the target vertex set (as filtered and processed by the full set of clauses in the SELECT query block) and assign that output set to the variable called result.

Last, we print out the result vertex set

### Create a query

Rather than defining our query in interactive mode, we can store the query in a file and invoke the file from within the GSQL shell, using the @filename syntax. Copy and paste the above query into a file /home/tigergraph/hello.gsql. Then, enter the GSQL shell and invoke the file using @hello.qsql.

In [12]:
results = conn.gsql(query)

print (results)

Using graph 'Social_101'
Semantic Check Fails: The query name hello is used by another object! Please use a different name.
Failed to create queries: [hello].


### Install a query

However, the query is not installed yet. Let's install query "hello".

It takes about 1 minute for the database to install this new query. Be patient! For queries on large datasets, this small investment pays off many times over in faster query execution, particularly if you will run the query many times, with different parameters. The installation will generate machine instructions and a REST endpoint. After the progress bar reaches 100%, we are ready to run this query.

In [13]:
results = conn.gsql(
'''
USE GRAPH Social_101

INSTALL QUERY hello
'''
)

print(results)

Using graph 'Social_101'
Semantic Check Fails: Graph Social_101: all queries in this catalog have been installed already.
Query installation finished.


### Run a query in GSQL

To run a query in GSQL, use RUN QUERY followed by the query name and a set of parameter values.

The result is presented in JSON format. Tom has two 1-hop neighbors, namely Dan and Jenny.

In [14]:
parameters = {
    "p": "Tom"
}

results = conn.runInstalledQuery("hello", params=parameters)
parsed = conn.parseQueryOutput(results)
print(json.dumps(parsed, indent=1))

{
 "vertices": {
  "Person": {
   "Dan": {
    "v_id": "Dan",
    "v_type": "Person",
    "attributes": {
     "name": "Dan",
     "age": 34,
     "gender": "male",
     "state": "ny"
    },
    "x_occurrences": 1,
    "x_sources": [
     "result"
    ]
   },
   "Jenny": {
    "v_id": "Jenny",
    "v_type": "Person",
    "attributes": {
     "name": "Jenny",
     "age": 25,
     "gender": "female",
     "state": "tx"
    },
    "x_occurrences": 1,
    "x_sources": [
     "result"
    ]
   }
  }
 },
 "edges": {}
}


### Query with accumulators

Now, let’s write a more advanced query. This time, we are going to learn to use the powerful built-in accumulators, which serve as the runtime attributes (properties) attachable to each vertex visited during our traversal on the graph.

"Runtime" means they exist only while the query is running; they are called accumulators because they are specially designed to gather (accumulate) data during the implicitly parallel processing of the query.

Say we need to write a query to find all the persons which are exactly 2 hops away from the parameterized input Person. Just for fun, let’s also compute the average age of those 2-hop neighbors.

#### Method

To get all Person vertices that are two hops away from the starting vertex, we can use the following logic:

From the starting vertex, visit vertices that are one stop away from the starting vertex, and mark every vertex we visit as visited, including the starting vertex. This gives us the neighbors that are 1 hop away from the starting vertex.

Now that we have the 1-hop neighbors, we can find vertices that are 1-hop away from those neighbors, excluding all vertices that we have already visited in the previous step. This gives us all vertices that are exactly two hops away from the original vertex. As we visit the 2-hop neighbors, we can also calculate their average age.

#### Query

We will use a SELECT statement to perform the first hop from the starting vertex. In order to keep track of vertices that have been visited, we declare a vertex-attached accumulator of the type OrAccum, also called a boolean accumulator. In addition, we also need to declare an AvgAccum to calculate the average age of the final vertex set.

To declare a vertex-attached accumulator, prefix an identifier name with a single @ symbol. After you declare the accumulator, they are attached to vertices in the query, and you can access its value in an ACCUM or POST-ACCUM clause of the SELECT statement by using the dot. operator following a vertex alias as if they are an attribute of the vertices.

After the first hop, we use another SELECT statement to perform a second hop, and filter out all vertices that we have visited previously. This gives us the neighbors that are exactly two hops away, and we can use the AvgAccum to calculate the average age of the 2-hop neighbors.

In [15]:
query2 = '''
USE GRAPH Social_101

CREATE QUERY hello2 (VERTEX<Person> p) {
    OrAccum  @visited = FALSE;
    AvgAccum @@avg_age;
    start = {p};

    first_neighbors = SELECT tgt
        FROM start:s -(Friendship:e)- Person:tgt
        ACCUM tgt.@visited += TRUE, s.@visited += TRUE;  

    second_neighbors = SELECT tgt 
        FROM first_neighbors -(:e)- :tgt 
        WHERE tgt.@visited == FALSE 
        POST-ACCUM @@avg_age += tgt.age; 

    PRINT second_neighbors;
    PRINT @@avg_age;
}

INSTALL QUERY hello2
'''

Note that a source vertex or target vertex may be visited multiple times in the first SELECT statement. Referring to Figure 1, if we start at vertex Tom, there are two edges incidental to Tom, so the ACCUM clause in the first SELECT statement will visit Tom twice. Since the accumulator type is OrAccum, the cumulative effect of the two traversals is the following:



Tom.@visited = (initial value: false) OR (true) OR (true)

We can see it does not matter which of the two edges was processed first. The net effect is that as long as a vertex is visited at least once, it will end up with @visited = true.



#### Run query

This time, we put all steps into one together

In [16]:
parameters = {
    "p": "Tom"
}

createAndInstall = conn.gsql(query2)
print (createAndInstall)

Using graph 'Social_101'
Semantic Check Fails: The query name hello2 is used by another object! Please use a different name.
Failed to create queries: [hello2].


In [17]:
results = conn.runInstalledQuery("hello2", params=parameters)
parsed = conn.parseQueryOutput(results)
print (json.dumps(parsed, indent=1))

{
 "vertices": {
  "Person": {
   "Amily": {
    "v_id": "Amily",
    "v_type": "Person",
    "attributes": {
     "name": "Amily",
     "age": 22,
     "gender": "female",
     "state": "ca",
     "@visited": false
    },
    "x_occurrences": 1,
    "x_sources": [
     "second_neighbors"
    ]
   },
   "Kevin": {
    "v_id": "Kevin",
    "v_type": "Person",
    "attributes": {
     "name": "Kevin",
     "age": 28,
     "gender": "male",
     "state": "az",
     "@visited": false
    },
    "x_occurrences": 1,
    "x_sources": [
     "second_neighbors"
    ]
   },
   "Nancy": {
    "v_id": "Nancy",
    "v_type": "Person",
    "attributes": {
     "name": "Nancy",
     "age": 20,
     "gender": "female",
     "state": "ky",
     "@visited": false
    },
    "x_occurrences": 1,
    "x_sources": [
     "second_neighbors"
    ]
   }
  }
 },
 "edges": {}
}


We can execute this full set of commands without entering the GSQL shell. Please copy and paste the above GSQL commands into a Linux file named /home/tigergraph/hello2.gsql.

In a Linux shell, under /home/tigergraph, type the following to create, install, and execute the query:

### GSQL query summary

* Queries are installed in the catalog and can have one or more input parameters, enabling reuse of queries.
* A GSQL query consists of a series of SELECT query blocks, each generating a named vertex set.
* Each SELECT query block can start traversing the graph from any of the previously defined vertex sets (that is, the sequence does not have to form a linear chain).
* Accumulators are runtime variables with built-in accumulation operations, for efficient multithreaded computation.
* Query can call another query.
* Output is in JSON format.